First we need to extrac data from sqlite database.

In [1]:
def get_data_from_file(db_file):
    import sqlite3
    
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        cur = conn.cursor()
        cur.execute("SELECT * FROM accelerometer_data")
        rows = cur.fetchall()
        conn.close()
        return rows
    except Error as e:
        print(e)
        return None

In [2]:
data = get_data_from_file('../accelerometer_data.db')

In [86]:
import pandas as pd
df = pd.DataFrame(data, columns = ['id', 'coor_x', 'coor_y', 'coor_z', 'timestamp', 'activity'])
df = df.drop(columns = ['id'])
df.head()

,coor_x,coor_y,coor_z,timestamp,activity
0,-0.951695,0.171185,9.878476,1574612512112,RUNNING
1,-0.954089,0.155623,9.865308,1574612512246,RUNNING
2,-0.958878,0.158017,9.860519,1574612512379,RUNNING
3,-0.957681,0.166397,9.859323,1574612512512,RUNNING
4,-0.950498,0.161609,9.861717,1574612512646,RUNNING


In [15]:
import sys
# sys.path.append('../') #the ROCKET project is in parent directory

from rocket_master.code.rocket_functions import generate_kernels, apply_kernels

In [ ]:
#normalise
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['coor_x', 'coor_y', 'coor_z']] = scaler.fit_transform(df[['coor_x', 'coor_y', 'coor_z']])
df[['coor_x', 'coor_y', 'coor_z']]

In [88]:
#split data
factor = 0.8
x_train = df[0:int(len(df) * factor)][['coor_x', 'coor_y', 'coor_z']]
y_train = df[0:int(len(df) * factor)][['activity']]
x_test = df[int(len(df) * factor):][['coor_x', 'coor_y', 'coor_z']]
y_test = df[int(len(df) * factor):][['activity']]

In [163]:
# (1) generate random kernels
kernels = generate_kernels(input_length = x_train.shape[1], num_kernels = 10_000)

In [165]:
kernels[0][100] #in form: weights, lengths, biases, dilations, paddings

array([-0.16777279, -0.04507909, -1.03198053, -0.08542557, -0.28644632,
        1.25098051,  0.45798593,  0.11767727, -0.20993942,  0.        ,
        0.        ])

In [178]:
x_train_transform = apply_kernels(x_train.to_numpy(), kernels)
x_train_transform[0]

20000

In [96]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV

classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10), normalize = True)
classifier.fit(x_train_transform, y_train.to_numpy())

# print(end = "") # suppress print output of classifier.fit(...)

d:\programy10\python\python37\lib\site-packages\sklearn\linear_model\ridge.py:1806: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  class_weight=None, cv=None, fit_intercept=True,
                  normalize=True, scoring=None, store_cv_values=False)

In [98]:
x_test_transform = apply_kernels(x_test.to_numpy(), kernels)

In [114]:
predictions = classifier.predict(x_test_transform)

print(f"predictions = {', '.join(predictions.astype(str))}")
print(f"accuracy    = {(predictions == y_test['activity']).mean()}") # or classifier.score(X_test_transform, Y_test)

predictions = WALKING, WALKING, RUNNING, WALKING, WALKING, WALKING, RUNNING, WALKING, RUNNING, RUNNING, RUNNING, RUNNING, RUNNING, RUNNING, RUNNING, RUNNING, WALKING, WALKING, WALKING, WALKING, WALKING, WALKING, STAYING, RUNNING, WALKING, WALKING, RUNNING, RUNNING, WALKING
accuracy    = 0.034482758620689655
